In [21]:
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""));


import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  
pd.set_option('display.width', 1000)

# Define

In [2]:
train_img_type = 'data_0_rotate'
train_dataset_path = f'{train_img_type}'
test_dataset_path = 'data'

In [3]:
import os
from collections import Counter

import numpy as np
import cv2

from matplotlib import pyplot as plt

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn import svm

from pycm import *

# config

In [4]:
normal_read = False
standard_scale = False
kernel = 'linear'
n_components = 60
normalize = False
ksize = 31

# Read data demo

In [5]:
list_category = sorted(os.listdir(test_dataset_path))

In [6]:
def read_img(path):
    if normal_read:
        img = np.array(cv2.imread(path))


        cond1 = np.logical_and((img[:,:,0]==255), (img[:,:,1]==255))
        cond2 = np.logical_and(cond1, (img[:,:,2]==255))

        target = np.zeros((img.shape[0], img.shape[1]))
        target[:,:] = np.where(cond2, 0, 1)
        
        return target
    else:
        img = cv2.imread(path)
        scale = 1
        delta = 0
        ddepth = cv2.CV_16S
        grad_x = cv2.Sobel(img, ddepth, 1, 0, ksize=ksize, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)
        grad_y = cv2.Sobel(img, ddepth, 0, 1, ksize=ksize, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)

        rs = np.zeros((img.shape[0], img.shape[1], 2))
        rs[:,:,0] = np.amax(grad_x,2)
        rs[:,:,1] = np.amax(grad_y,2)
        return np.amax(rs, 2)

# Now split data by (category & id) for faster looping

In [7]:
train_list = []
val_list = []

train_ratio = 0.7

for cat in list_category:
    list_file_by_cat = os.listdir(os.path.join(test_dataset_path, cat))
    train_length = int(len(list_file_by_cat) * train_ratio)
    
    for fn in list_file_by_cat[:train_length]:
        train_list.append((cat, fn.replace('.png', '')))
        
    for fn in list_file_by_cat[train_length:]:
        val_list.append((cat, fn.replace('.png', '')))

Now reading data to train/test set

In [8]:
from sklearn.preprocessing import StandardScaler
def flatten_data(img):
    if standard_scale:
        scaler = StandardScaler()
        img = scaler.fit_transform(img)
    return [item for sublist in img for item in sublist]

In [9]:
category_mapping = {}

for i in range(len(list_category)):
    category_mapping[list_category[i]] = i

In [10]:
train_x = []
train_y = []

for cat, fn in tqdm(train_list):
    fn_dir = os.path.join(train_dataset_path, cat, fn)
    for file in os.listdir(fn_dir):
        train_x.append(flatten_data(read_img(os.path.join(fn_dir, file))))
        train_y.append(category_mapping[cat])

100%|██████████| 14000/14000 [06:26<00:00, 36.24it/s] 


In [11]:
val_x = []
val_y = []

for cat, fn in tqdm(val_list):
    val_x.append(flatten_data(read_img(os.path.join(test_dataset_path, cat, f'{fn}.png'))))
    val_y.append(category_mapping[cat])

100%|██████████| 6000/6000 [02:43<00:00, 36.62it/s]


# Train & Report

In [12]:
import snapml

In [13]:
model = snapml.SupportVectorMachine(
    kernel=kernel,
    n_jobs=30,
    n_components=n_components,
    normalize=normalize,
    verbose=True,
)

In [14]:
model.fit(np.array(train_x), train_y)

[Info] Training will run in multi-threaded mode on CPU.


SupportVectorMachine(device_ids=array([], dtype=uint32), n_components=60,
                     n_jobs=30, verbose=True)

In [15]:
top_1 = 0
top_3 = 0
top_10 = 0

In [16]:
i = 0
for i in tqdm(range(len(val_x))):
    x = np.array(val_x[i:i+1])
    rs = model.decision_function(x, n_jobs=10)[0]
    top_rs = rs.argsort()[-10:][::-1]
    if val_y[i] == top_rs[0]:
        top_1 += 1
    if val_y[i] in top_rs[:3]:
        top_3 += 1
    if val_y[i] in top_rs:
        top_10 += 1
    i += 1

100%|██████████| 6000/6000 [18:59<00:00,  5.27it/s]  


In [17]:
print(top_1)
print(top_3)
print(top_10)

1009
1704
2686


In [18]:
import pickle

In [19]:
with open('model_2.pkl', "wb") as f:
    pickle.dump(model, f)

In [20]:
# y_pred = model.predict(np.array(val_x))
# cm = ConfusionMatrix(actual_vector=val_y, predict_vector=list(y_pred.astype('int')))
# cm.save_html(f'{train_img_type}')